In [1]:
import xarray as xr
import sys
import pandas as pd
import numpy as np
import xesmf as xe
import warnings
import dask
import datetime

warnings.filterwarnings('ignore')

In [2]:
# get the workers going
#ncores = 36
#nmem = str(int(365*ncores/36))+'GB'
#from dask_jobqueue import SLURMCluster
#from dask.distributed import Client
#cluster = SLURMCluster(cores=ncores,
#                     processes=ncores, memory=nmem,
#                     project='P04010022',
#                     walltime='12:00:00')
#cluster.scale(ncores)
#client = Client(cluster)

In [6]:
# get the workers going
ncores = 1
nworkers = 12
nmems="5GB"
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(
cores = ncores, processes = ncores, project="P04010022", walltime="04:00:00")
cluster.scale(nworkers)
client = Client(cluster)

In [9]:
# do this until you see you've got some workers
client

Client Scheduler: tcp://10.12.205.27:40171 Dashboard: http://10.12.205.27/proxy/35489/status,Cluster Workers: 12 Cores: 12 Memory: 300.00 GB


In [10]:
# location of ERA5 data on RDA
filepath="/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu/"
# output location
outpath="/glade/scratch/islas/processed/era5/STRD_day/"

In [11]:
ystart=1979 ; yend=2019 ; nyears=yend-ystart+1

In [12]:
# open up CESM data to get the output grid.
cesmdat = xr.open_dataset("/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/PHIS/f.e11.F1850C5CNTVSST.f09_f09.002.cam.h0.PHIS.040101-050012.nc")
grid_out = xr.Dataset({'lat': (['lat'], cesmdat.lat)}, {'lon': (['lon'], cesmdat.lon)})

In [133]:
dailydat2.STRD.data

dask.array<concatenate, shape=(721, 1440, 31), dtype=float32, chunksize=(721, 1440, 15), chunktype=numpy.ndarray>

In [13]:
reusewgt=False
wgtfile=outpath+"wgtfile.nc"
for iyear in range(ystart,yend+1,1):
    print(iyear)
    timeout = pd.date_range(start = str(iyear)+"-01-01", end = str(iyear)+"-12-31")
    
    countdays=0
    for imon in range(1,12+1,1):
    #for imon in range(2,3,1):
        monstr=str(imon).zfill(2)
        outfile=outpath+"strd_"+str(iyear)+monstr+".nc"
        file=filepath+"/"+str(iyear)+monstr+"/*_str.*.nc"
        print(file)
        data = xr.open_mfdataset(file,coords="minimal", join="override",
                             decode_times=True, use_cftime=True )
        data = data.STR
        data6h = data.loc[{"forecast_initial_time": data.forecast_initial_time.dt.hour.isin([6])}]
        data18h = data.loc[{"forecast_initial_time": data.forecast_initial_time.dt.hour.isin([18])}]
        
        middle = data6h.sum(dim="forecast_hour")
        end = data18h.loc[dict(forecast_hour=slice(1,6))].sum(dim="forecast_hour")
        beginning = data18h.loc[dict(forecast_hour=slice(7,13))].sum(dim="forecast_hour")
        
        dayspermon = middle.forecast_initial_time.dt.daysinmonth.data
        dayendstr = str(dayspermon[0])
        timeout = pd.date_range(
            start=str(iyear)+"-"+monstr+"-01",
            end=str(iyear)+"-"+monstr+"-"+dayendstr)
        timeout2 = timeout + datetime.timedelta(days=1)
        beginning['forecast_initial_time'] = timeout2
        middle['forecast_initial_time'] = timeout
        end['forecast_initial_time'] = timeout
        
        dailydat = (beginning + middle + end)/86400.
        if ((imon == 1) and (iyear == ystart)):
            day1dat = (middle.isel(forecast_initial_time=0) + end.isel(forecast_initial_time=0))/64800.
        else:
            day1dat = (begsaved + middle.isel(forecast_initial_time=0) + end.isel(forecast_initial_time=0))/86400.
        
        dailydat2 = xr.concat([day1dat, dailydat], dim="forecast_initial_time")
        begsaved = beginning.isel(forecast_initial_time=dayspermon[0]-1).persist()
        
        #dailydat2 = dailydat2.rename(longitude="lon")
        #dailydat2 = dailydat2.rename(latitude="lat")

        #dailydat2 = dailydat2.chunk({'forecast_initial_time': -1})
        dailydat2 = dailydat2.transpose("forecast_initial_time","latitude","longitude")
        regridder = xe.Regridder(dailydat2, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
        reusewgt=True
        dataday_rg = regridder(dailydat2)
        dataday_rg = dataday_rg.rename(forecast_initial_time="time")
        dataday_rg = dataday_rg.assign_attrs({'long_name':'Surface net thermal longwave radiation', 'units':'Wm**2'})

        dataday_rg.to_netcdf(path=outfile)

1979
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197901/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197902/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197903/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197904/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197905/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197906/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197907/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197908/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197909/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197910/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197911/*_str.*.nc
/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.fc.sfc.accumu//197912/*_str.*.nc
1980
/gpfs/fs1/collections/rda/data